## Preprocessing

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot
import seaborn as sns
# Too see full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import urllib.request # library to handle requests
import requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup #webscrapping tools
print('Libraries imported.')

Libraries imported.


In [3]:
# Link To Extract, 
website_url = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# Read File
soup = BeautifulSoup(website_url,'lxml')
right_table=soup.find('table', class_='wikitable sortable')
# extract the content string from within each <td> to get the data
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3 and cells[1].find(text=True).strip() != 'Not assigned':
        A.append(cells[0].find(text=True).strip())
        B.append(cells[1].find(text=True).strip())
        C.append(cells[2].find(text=True).strip())

df = pd.DataFrame(A, columns=['Postal Code'])
df['Borough'] = B
df['Neighborhood'] = C
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
# read geospatial data
df_loc = pd.read_csv('Geospatial_Coordinates.csv')

# merge dataframe
df_toronto = pd.merge(left = df, right = df_loc, how='left', left_on='Postal Code', right_on='Postal Code')

In [5]:
# Sorting
# set index for only Downtown Toronto
downtown_toronto_data = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
# eliminate 'Postcode' column
downtown_toronto_data=downtown_toronto_data.drop(['Postal Code'], axis=1)
# cek data
downtown_toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


### Now we will move towards New York Boroughs. We select "Manhattan" as a Borough and anylze its neighborhoods later

In [6]:
url = 'https://cocl.us/new_york_dataset'
filename = 'newyork_data.json'
urllib.request.urlretrieve(url, filename)

('newyork_data.json', <http.client.HTTPMessage at 0x2566c481188>)

In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [8]:
# retrieve data from json
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# Then let's loop through the data and fill the dataframe one row at a time.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [10]:
# Creating new Dataframe manhattan_data
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


## FOURSQUARE API

In [11]:
CLIENT_ID = '4WWJDXGAJXZEVH1HI4A1CDQWXNWCIRIHWOAHKK3S21IBDOYP' # your Foursquare ID
CLIENT_SECRET = 'BNW0A1L05HYVCIGQ4CQ50JYRFE0GJU33BQ2V31JWLZT4RMQL' # your Foursquare Secret
VERSION = '20200605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4WWJDXGAJXZEVH1HI4A1CDQWXNWCIRIHWOAHKK3S21IBDOYP
CLIENT_SECRET:BNW0A1L05HYVCIGQ4CQ50JYRFE0GJU33BQ2V31JWLZT4RMQL


In [12]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [13]:
# Let's get the geographical coordinates of Manhattan.
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


## VISUALIZATION

We visualize the data many times at different stages. In the beginning, we visualize the selected borough neighborhoods so that we can get an idea or confirmation regarding the coordinates of that Borough. The second time after clustered the neighborhoods, we visualize the clusters to name them. Assigning the names are very important because it can identify the areas or specific places in each cluster

### (Before Clustering)

#### DOWNTOWN TORONTO

In [14]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=13)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [15]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=13)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto


#### MANHATTAN

In [16]:
# let's visualizat Manhattan the neighborhoods in it.
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [17]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

grouping = plugins.MarkerCluster().add_to(map_manhattan)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(grouping)  
    
map_manhattan

### ANALYSIS

We analyze both boroughs neighborhoods through one hot encoding (giving ‘1’ if a venue category is there, and ‘0’ in case of venue category is not there). On the basis of one hot encoding, we calculate mean of the frequency of occurrence of each category and picked top ten venues on that basis for each neighborhood. It means the top venues are showing the foot traffic or the more visited places.

In [18]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
limit = 20
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Exploring Neighborhoods in Downtown Toronto

In [19]:
# Write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'],
                                  )

In [20]:
# Let's check the size of the resulting dataframe
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(357, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [21]:
# Let's check how many venues were returned for each neighborhood
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,20,20,20,20,20,20
Christie,18,18,18,18,18,18
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
"First Canadian Place, Underground city",20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20
"Harbourfront East, Union Station, Toronto Islands",20,20,20,20,20,20


In [22]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 125 uniques categories.


### Analyzing Each Neighborhood in Downtown Toronto

In [24]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Farmers Market,Fish Market,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Museum,Music Venue,Neighborhood,Nightclub,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [26]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.05
1  Liquor Store  0.05
2   Fish Market  0.05
3        Museum  0.05
4      Fountain  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.13
1      Airport Service  0.13
2     Airport Terminal  0.13
3  Rental Car Location  0.07
4                  Bar  0.07


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.25
1  Japanese Restaurant  0.10
2            Gastropub  0.05
3   Miscellaneous Shop  0.05
4                  Spa  0.05


----Christie----
           venue  freq
0  Grocery Store  0.22
1           Café  0.17
2           Park  0.11
3    Coffee Shop  0.11
4          Diner  0.06


----Church and Wellesley----
                venue  freq
0           Wine Shop  0.05
1            Creperie  0.05
2     Bubble Tea Shop  0.05
3        Burger Joint  0.05
4  Salon / Barbershop  0.0

In [27]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Greek Restaurant,Bakery,Cocktail Bar,Museum,Coffee Shop,Liquor Store,Concert Hall,Breakfast Spot,Restaurant,Jazz Club
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina,Rental Car Location,Sculpture Garden,Boat or Ferry,Bar,Coffee Shop
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Gastropub,Sushi Restaurant,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Ramen Restaurant,Bubble Tea Shop,Comic Shop
3,Christie,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Nightclub,Candy Store,Restaurant,Baby Store,Italian Restaurant
4,Church and Wellesley,Wine Shop,Beer Bar,Salon / Barbershop,Breakfast Spot,Restaurant,Bubble Tea Shop,Ramen Restaurant,Burger Joint,General Entertainment,Park
5,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Tea Room,Museum,Pub,Restaurant,Bookstore,Bakery,Ice Cream Shop,Japanese Restaurant
6,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Pizza Place,Steakhouse,Gluten-free Restaurant,Gym,Gym / Fitness Center,Pub,Bookstore
7,"Garden District, Ryerson",Café,Pizza Place,Mexican Restaurant,Sandwich Place,College Rec Center,Music Venue,Ramen Restaurant,Sporting Goods Shop,Steakhouse,Diner
8,"Harbourfront East, Union Station, Toronto Islands",Hotel,Park,Café,Plaza,Skating Rink,Bubble Tea Shop,Italian Restaurant,Deli / Bodega,Sporting Goods Shop,Ice Cream Shop
9,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Gourmet Shop,Bakery,Coffee Shop,Cocktail Bar,Cheese Shop,Caribbean Restaurant,Mexican Restaurant,Dessert Shop


### Clustering Neighborhoods

In [29]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 3, 0, 3, 3, 4, 4, 3])

In [30]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = downtown_toronto_data

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Breakfast Spot,Park,Farmers Market,Restaurant,Spa,Dessert Shop,Bakery,Distribution Center,Performing Arts Venue
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Arts & Crafts Store,Burrito Place,Mexican Restaurant,Creperie,Beer Bar,Italian Restaurant
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Café,Pizza Place,Mexican Restaurant,Sandwich Place,College Rec Center,Music Venue,Ramen Restaurant,Sporting Goods Shop,Steakhouse,Diner
3,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Gastropub,Japanese Restaurant,Coffee Shop,Hotel,Poke Place,Middle Eastern Restaurant,Restaurant,Cosmetics Shop,Creperie,Italian Restaurant
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Greek Restaurant,Bakery,Cocktail Bar,Museum,Coffee Shop,Liquor Store,Concert Hall,Breakfast Spot,Restaurant,Jazz Club


In [32]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

#### CLuster 1 - Cafes

In [34]:
downtown_toronto_merged.loc[
    downtown_toronto_merged['Cluster Labels'] == 0, 
    downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Park,Farmers Market,Restaurant,Spa,Dessert Shop,Bakery,Distribution Center,Performing Arts Venue
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Arts & Crafts Store,Burrito Place,Mexican Restaurant,Creperie,Beer Bar,Italian Restaurant
4,Berczy Park,Greek Restaurant,Bakery,Cocktail Bar,Museum,Coffee Shop,Liquor Store,Concert Hall,Breakfast Spot,Restaurant,Jazz Club
12,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Gourmet Shop,Bakery,Coffee Shop,Cocktail Bar,Cheese Shop,Caribbean Restaurant,Mexican Restaurant,Dessert Shop
16,"St. James Town, Cabbagetown",Café,Restaurant,Gastropub,Indian Restaurant,Park,Pet Store,General Entertainment,Caribbean Restaurant,Butcher,Pub


#### Cluster 2 - Airport

In [35]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, 
                            downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina,Rental Car Location,Sculpture Garden,Boat or Ferry,Bar,Coffee Shop


#### Cluster 3 - Restaurants, Cafes

In [37]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, 
                            downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Garden District, Ryerson",Café,Pizza Place,Mexican Restaurant,Sandwich Place,College Rec Center,Music Venue,Ramen Restaurant,Sporting Goods Shop,Steakhouse,Diner
10,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Tea Room,Museum,Pub,Restaurant,Bookstore,Bakery,Ice Cream Shop,Japanese Restaurant
11,"University of Toronto, Harbord",Restaurant,Bookstore,Bakery,Japanese Restaurant,Yoga Studio,Italian Restaurant,College Gym,Comfort Food Restaurant,Café,Sandwich Place
14,Rosedale,Park,Playground,Trail,Deli / Bodega,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center


#### Cluster 4 - Gastropub, Asian Resto, and cafes

In [40]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, 
                            downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,St. James Town,Gastropub,Japanese Restaurant,Coffee Shop,Hotel,Poke Place,Middle Eastern Restaurant,Restaurant,Cosmetics Shop,Creperie,Italian Restaurant
5,Central Bay Street,Coffee Shop,Japanese Restaurant,Gastropub,Sushi Restaurant,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Ramen Restaurant,Bubble Tea Shop,Comic Shop
6,Christie,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Nightclub,Candy Store,Restaurant,Baby Store,Italian Restaurant
9,"Toronto Dominion Centre, Design Exchange",Café,Coffee Shop,Steakhouse,Pizza Place,Pub,Restaurant,Japanese Restaurant,Sandwich Place,Beer Bar,Bakery
17,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Pizza Place,Steakhouse,Gluten-free Restaurant,Gym,Gym / Fitness Center,Pub,Bookstore


#### Cluster 5 - Hotel, Coffee shop, club

In [42]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, 
                            downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"Richmond, Adelaide, King",Coffee Shop,Seafood Restaurant,Restaurant,Concert Hall,Bar,Hotel,Smoke Shop,Plaza,Speakeasy,Steakhouse
8,"Harbourfront East, Union Station, Toronto Islands",Hotel,Park,Café,Plaza,Skating Rink,Bubble Tea Shop,Italian Restaurant,Deli / Bodega,Sporting Goods Shop,Ice Cream Shop
15,Stn A PO Boxes,Cocktail Bar,Café,Park,Art Gallery,Jazz Club,Seafood Restaurant,Comfort Food Restaurant,Hotel,Concert Hall,Tailor Shop
18,Church and Wellesley,Wine Shop,Beer Bar,Salon / Barbershop,Breakfast Spot,Restaurant,Bubble Tea Shop,Ramen Restaurant,Burger Joint,General Entertainment,Park


### Exploring Neighborhood in Manhattan

In [45]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'],
                                  )

In [50]:
# Let's check how many venues were returned for each neighborhood
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,20,20,20,20,20,20
Carnegie Hill,20,20,20,20,20,20
Central Harlem,20,20,20,20,20,20
Chelsea,20,20,20,20,20,20
Chinatown,20,20,20,20,20,20
Civic Center,20,20,20,20,20,20
Clinton,20,20,20,20,20,20
East Harlem,20,20,20,20,20,20
East Village,20,20,20,20,20,20


In [51]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 205 uniques categories.


#### Analyzing The Neighborhoods

In [53]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Heliport,Himalayan Restaurant,Historic Site,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Noodle House,Opera House,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [55]:
# Set Index
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

In [56]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0            Park  0.15
1   Memorial Site  0.15
2      Food Court  0.10
3           Plaza  0.05
4  Scenic Lookout  0.05


----Carnegie Hill----
                  venue  freq
0                   Gym  0.10
1           Coffee Shop  0.10
2  Gym / Fitness Center  0.10
3    Italian Restaurant  0.10
4           Pizza Place  0.05


----Central Harlem----
                  venue  freq
0   American Restaurant  0.10
1     French Restaurant  0.10
2             Juice Bar  0.05
3              Beer Bar  0.05
4  Ethiopian Restaurant  0.05


----Chelsea----
                venue  freq
0      Ice Cream Shop  0.10
1             Theater  0.10
2                 Bar  0.05
3  Seafood Restaurant  0.05
4   French Restaurant  0.05


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.15
1      Sandwich Place  0.10
2           Bike Shop  0.05
3     Bubble Tea Shop  0.05
4              Museum  0.05


----Civic Center----
                   venue

In [57]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Memorial Site,Food Court,BBQ Joint,Sandwich Place,Gym,Shopping Mall,Scenic Lookout,Plaza,Performing Arts Venue
1,Carnegie Hill,Coffee Shop,Gym,Italian Restaurant,Gym / Fitness Center,Café,Wine Shop,Wine Bar,Pizza Place,American Restaurant,Dance Studio
2,Central Harlem,French Restaurant,American Restaurant,Gym / Fitness Center,Boutique,Food Truck,Ethiopian Restaurant,Cycle Studio,Beer Bar,Juice Bar,Bar
3,Chelsea,Ice Cream Shop,Theater,Beer Bar,Hotel,Italian Restaurant,Market,French Restaurant,Fish Market,New American Restaurant,Coffee Shop
4,Chinatown,Chinese Restaurant,Sandwich Place,English Restaurant,Bike Shop,Noodle House,Tea Room,Greek Restaurant,New American Restaurant,Cocktail Bar,Bakery
5,Civic Center,Spa,Gym / Fitness Center,Yoga Studio,Taco Place,Dance Studio,Cuban Restaurant,Park,Falafel Restaurant,Nail Salon,Burrito Place
6,Clinton,Theater,Gym / Fitness Center,Hotel Bar,Building,Movie Theater,Pie Shop,Sporting Goods Shop,Sports Bar,Mediterranean Restaurant,Comedy Club
7,East Harlem,Mexican Restaurant,Thai Restaurant,French Restaurant,Cuban Restaurant,Doctor's Office,Park,Donut Shop,Sandwich Place,New American Restaurant,Spanish Restaurant
8,East Village,Wine Bar,Dessert Shop,Vietnamese Restaurant,Speakeasy,Bagel Shop,Moroccan Restaurant,Scandinavian Restaurant,Korean Restaurant,Beer Store,Park
9,Financial District,Coffee Shop,Event Space,Gym / Fitness Center,Salad Place,Restaurant,Spa,Monument / Landmark,Steakhouse,New American Restaurant,Falafel Restaurant


### Clustering The Neighborhood

Now we applied Machine Learning Technique “Clustering” to segment the neighborhoods in similar objects cluster. This will help to analyze from Tourist perspective and we can easily extract the Tourist places which are present on one of the clusters.

In [58]:
# Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 2, 0, 4, 2, 2, 1])

In [59]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Gym,Coffee Shop,Sandwich Place,Yoga Studio,Tennis Stadium,Miscellaneous Shop,Donut Shop,Discount Store,Diner,Department Store
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Sandwich Place,English Restaurant,Bike Shop,Noodle House,Tea Room,Greek Restaurant,New American Restaurant,Cocktail Bar,Bakery
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Park,Wine Shop,Market,Coffee Shop,Cocktail Bar,Restaurant,Ramen Restaurant,Frozen Yogurt Shop,Bakery
3,Manhattan,Inwood,40.867684,-73.921210,0,Bakery,Park,Yoga Studio,Deli / Bodega,Diner,Restaurant,Farmers Market,Café,Mexican Restaurant,Pharmacy
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Yoga Studio,Mexican Restaurant,Cocktail Bar,Burger Joint,Italian Restaurant,Japanese Restaurant,Historic Site,Mediterranean Restaurant,Pizza Place,Coffee Shop


In [60]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster

#### Cluster 1 - Restaurants and cafes

In [62]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Gym,Coffee Shop,Sandwich Place,Yoga Studio,Tennis Stadium,Miscellaneous Shop,Donut Shop,Discount Store,Diner,Department Store
2,Washington Heights,Café,Park,Wine Shop,Market,Coffee Shop,Cocktail Bar,Restaurant,Ramen Restaurant,Frozen Yogurt Shop,Bakery
3,Inwood,Bakery,Park,Yoga Studio,Deli / Bodega,Diner,Restaurant,Farmers Market,Café,Mexican Restaurant,Pharmacy
5,Manhattanville,Mexican Restaurant,Italian Restaurant,Seafood Restaurant,Bike Trail,Coffee Shop,Ramen Restaurant,Gastropub,Juice Bar,Bar,Lounge
10,Lenox Hill,Thai Restaurant,Gym,Health Food Store,Burger Joint,Dessert Shop,Cocktail Bar,College Academic Building,Gift Shop,Salad Place,Liquor Store
12,Upper West Side,American Restaurant,Italian Restaurant,Mediterranean Restaurant,Bar,Southern / Soul Food Restaurant,Seafood Restaurant,Movie Theater,Nail Salon,Bookstore,Greek Restaurant
14,Clinton,Theater,Gym / Fitness Center,Hotel Bar,Building,Movie Theater,Pie Shop,Sporting Goods Shop,Sports Bar,Mediterranean Restaurant,Comedy Club
23,Soho,Women's Store,Boutique,Men's Store,Clothing Store,Supermarket,Dance Studio,Electronics Store,Salon / Barbershop,Miscellaneous Shop,Sporting Goods Shop
25,Manhattan Valley,Bar,Yoga Studio,Bubble Tea Shop,Indian Restaurant,Italian Restaurant,Hostel,Hawaiian Restaurant,Korean Restaurant,Grocery Store,Mexican Restaurant
26,Morningside Heights,Bookstore,Park,Sandwich Place,American Restaurant,Coffee Shop,Burger Joint,Café,Salad Place,Farmers Market,Outdoor Sculpture


#### Cluster 2 - Public space (parks, spa etc)

In [63]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Sandwich Place,English Restaurant,Bike Shop,Noodle House,Tea Room,Greek Restaurant,New American Restaurant,Cocktail Bar,Bakery
9,Yorkville,Wine Shop,Deli / Bodega,Coffee Shop,Café,Sandwich Place,Gym,Beer Store,Diner,Park,Bagel Shop
22,Little Italy,Sandwich Place,Ice Cream Shop,Wine Bar,Coffee Shop,Snack Place,Salon / Barbershop,Gourmet Shop,Pizza Place,Women's Store,Chinese Restaurant
27,Gramercy,Pizza Place,Coffee Shop,Yoga Studio,Spa,American Restaurant,Beer Bar,Bike Rental / Bike Share,Cocktail Bar,Comedy Club,Filipino Restaurant
28,Battery Park City,Park,Memorial Site,Food Court,BBQ Joint,Sandwich Place,Gym,Shopping Mall,Scenic Lookout,Plaza,Performing Arts Venue
37,Stuyvesant Town,Park,Playground,Skating Rink,Gas Station,Farmers Market,Cocktail Bar,Bistro,Coffee Shop,Gym / Fitness Center,Baseball Field
39,Hudson Yards,Hotel,American Restaurant,Gym / Fitness Center,Pet Store,Public Art,Building,Music School,Furniture / Home Store,Cocktail Bar,Supermarket


#### Cluster 3 Hotel, theater, restaurant

In [65]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Yoga Studio,Mexican Restaurant,Cocktail Bar,Burger Joint,Italian Restaurant,Japanese Restaurant,Historic Site,Mediterranean Restaurant,Pizza Place,Coffee Shop
7,East Harlem,Mexican Restaurant,Thai Restaurant,French Restaurant,Cuban Restaurant,Doctor's Office,Park,Donut Shop,Sandwich Place,New American Restaurant,Spanish Restaurant
8,Upper East Side,Hotel,Italian Restaurant,Hotel Bar,Bookstore,Coffee Shop,Salad Place,Seafood Restaurant,Optical Shop,Cosmetics Shop,Bar
11,Roosevelt Island,Playground,Greek Restaurant,Cosmetics Shop,Park,Outdoors & Recreation,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Coffee Shop,School
13,Lincoln Square,Theater,Performing Arts Venue,Concert Hall,Indie Movie Theater,Gym / Fitness Center,Plaza,Circus,College Arts Building,Fountain,Library
15,Midtown,Hotel,Cuban Restaurant,Bookstore,Sporting Goods Shop,Spa,Smoke Shop,Clothing Store,Salad Place,Park,Szechuan Restaurant
16,Murray Hill,Hotel,Japanese Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Ramen Restaurant,Tea Room,Hawaiian Restaurant,Jewish Restaurant,Coffee Shop,Sushi Restaurant
18,Greenwich Village,Italian Restaurant,French Restaurant,Clothing Store,Sushi Restaurant,Café,Beer Bar,Snack Place,Pilates Studio,Caribbean Restaurant,Food Truck
19,East Village,Wine Bar,Dessert Shop,Vietnamese Restaurant,Speakeasy,Bagel Shop,Moroccan Restaurant,Scandinavian Restaurant,Korean Restaurant,Beer Store,Park
20,Lower East Side,Coffee Shop,Art Gallery,Cocktail Bar,Yoga Studio,Mediterranean Restaurant,Ice Cream Shop,Performing Arts Venue,Ramen Restaurant,Filipino Restaurant,Clothing Store


#### Cluster 4 - Coffee shop

In [67]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Financial District,Coffee Shop,Event Space,Gym / Fitness Center,Salad Place,Restaurant,Spa,Monument / Landmark,Steakhouse,New American Restaurant,Falafel Restaurant


#### Cluster 5 Restaurants, Theater

In [68]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, 
                     manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Harlem,French Restaurant,American Restaurant,Gym / Fitness Center,Boutique,Food Truck,Ethiopian Restaurant,Cycle Studio,Beer Bar,Juice Bar,Bar
17,Chelsea,Ice Cream Shop,Theater,Beer Bar,Hotel,Italian Restaurant,Market,French Restaurant,Fish Market,New American Restaurant,Coffee Shop


## Results

After clustering the data of the respective neighborhoods, both cities (Boroughs) have venues which can be explored and attract the Tourists. The neighborhoods are much similar in features like Theaters, opera houses, food places, clubs, museums, parks etc. As far as concern to dissimilarity, it differs in terms of some unique places like historical places and monuments

### Observations & Recommendations

When we compare the tourist places, we observe that the historical place is only situated in Downtown Toronto and the Monument or landmark venue is in Manhattan neighborhoods. Similarly, Airport facility, Harbor, Sculpture garden and Boat or ferry services are also available in Downtown Toronto while venues like Nightlife, Climbing gym and Museums are present in Manhattan.


As far as concern to recommendations, we recommend Downtown Toronto Neighborhoods will be considered first to visit. The tourists have an easily travelling access due to Airport facility, which not only saves time but also helps to save money. This saved money can be utilized to explore more, the attracting venues.

## Conclusions

The Downtown Toronto and Manhattan neighborhoods have more like similar venues. As we know that every place is unique in its own way, so that’s argument is present in both neighborhoods. The dissimilarity exists in terms of some different venues and facilities but not on a larger extent.